In [1]:
pip install xgboost lightgbm catboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 10.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 16.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [106]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor 
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'March-Mania-2023/Evan-Data/MNCAA_train_538.csv'
file_key_2 = 'March-Mania-2023/Evan-Data/MNCAA_test_538.csv'
file_key_3 = 'March-Mania-2023/Evan-Data/WNCAA_train_538.csv'
file_key_4 = 'March-Mania-2023/Evan-Data/WNCAA_test_538.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data files
man_train = pd.read_csv(file_content_stream_1)
man_test = pd.read_csv(file_content_stream_2)
woman_train = pd.read_csv(file_content_stream_3)
woman_test = pd.read_csv(file_content_stream_4)

/tmp/ipykernel_15056/467976974.py:50: DtypeWarning: Columns (46,55) have mixed types. Specify dtype option on import or set low_memory=False.
  man_test = pd.read_csv(file_content_stream_2)
/tmp/ipykernel_15056/467976974.py:52: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  woman_test = pd.read_csv(file_content_stream_4)


# Man

In [102]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'Seed1',
             'Seed2',
             'SeedDiff',
             'quality_march_T1',
             'quality_march_T2']

X_train = man_train[to_select]
Y_train = man_train['ResultDiff']

man_test_tour =  man_test[~(man_test['Seed1'].isnull() | man_test['Seed2'].isnull())].reset_index(drop = True)
man_test_IDS = man_test_tour['ID'] 
man_test_tour = man_test_tour[to_select]

In [103]:
#############
## XGBoost ##
#############

xgb_params = pd.read_csv('man_XGB_Phase_1_42_Optuna_Hyperparameters.csv')

xgb_md = XGBRegressor(tree_method = 'hist', 
                      max_depth = xgb_params['max_depth'][0],
                      learning_rate = xgb_params['learning_rate'][0],
                      n_estimators = xgb_params['n_estimators'][0],
                      gamma = xgb_params['gamma'][0],
                      min_child_weight = xgb_params['min_child_weight'][0],
                      colsample_bytree = xgb_params['colsample_bytree'][0],
                      subsample = xgb_params['subsample'][0]).fit(X_train, Y_train)

xgb_pred = xgb_md.predict(man_test_tour)

##############
## LightGBM ##
##############

lgb_params = pd.read_csv('man_LightGBM_Phase_1_42_Optuna_Hyperparameters.csv')

lgb_md = LGBMRegressor(boosting_type = 'gbdt', 
                       n_estimators = lgb_params['n_estimators'][0],
                       learning_rate = lgb_params['learning_rate'][0],
                       max_depth = lgb_params['max_depth'][0],
                       lambda_l1 = lgb_params['lambda_l1'][0],
                       lambda_l2 = lgb_params['lambda_l2'][0],
                       num_leaves = lgb_params['num_leaves'][0],
                       bagging_fraction = lgb_params['bagging_fraction'][0],
                       feature_fraction = lgb_params['feature_fraction'][0]).fit(X_train, Y_train)

lgb_pred = lgb_md.predict(man_test_tour)

##################
## HistGradient ##
##################

hist_params = pd.read_csv('man_Hist_Phase_1_42_Optuna_Hyperparameters.csv')

hist_md = HistGradientBoostingRegressor(l2_regularization = hist_params['l2_regularization'][0],
                                        early_stopping = False,
                                        learning_rate = hist_params['learning_rate'][0],
                                        max_iter = hist_params['max_iter'][0],
                                        max_depth = hist_params['max_depth'][0],
                                        max_bins = hist_params['max_bins'][0],
                                        min_samples_leaf = hist_params['min_samples_leaf'][0],
                                        max_leaf_nodes = hist_params['max_leaf_nodes'][0]).fit(X_train, Y_train)

hist_pred = hist_md.predict(man_test_tour)

##############
## Ensemble ##
##############

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
man_test_tour['ResultDiff'] = np.round_(ens_pred, decimals = 0).astype(int)
man_test_tour['ID'] = man_test_IDS

[LightGBM] [Warning] lambda_l1 is set=0.1713152100541493, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1713152100541493
[LightGBM] [Warning] feature_fraction is set=0.2439888911439529, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2439888911439529
[LightGBM] [Warning] lambda_l2 is set=1.290608460937753, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.290608460937753
[LightGBM] [Warning] bagging_fraction is set=0.8443456049471257, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8443456049471257


In [104]:
data_538 = ['ID',
            'X1_team_rating',
            'X1_rd1_win',
            'X1_rd2_win',
            'X1_rd3_win',
            'X1_rd4_win',
            'X1_rd5_win',
            'X1_rd6_win',
            'X1_rd7_win',
            'X2_team_rating',
            'X2_rd1_win',
            'X2_rd2_win',
            'X2_rd3_win',
            'X2_rd4_win',
            'X2_rd5_win',
            'X2_rd6_win',
            'X2_rd7_win']

man_test_538 = man_test[data_538]

man_test_tour = pd.merge(man_test_tour, man_test_538, on = 'ID', how = 'left')
man_test_tour.head()

,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2,ResultDiff,ID,X1_team_rating,X1_rd1_win,X1_rd2_win,X1_rd3_win,X1_rd4_win,X1_rd5_win,X1_rd6_win,X1_rd7_win,X2_team_rating,X2_rd1_win,X2_rd2_win,X2_rd3_win,X2_rd4_win,X2_rd5_win,X2_rd6_win,X2_rd7_win
0,0.8,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.52876,75.384559,1.884614,0.275133,0.17941,0.740689,0.800000,82.676471,84.5,11.176471,59.264706,59.0,49,71,19.176471,3.205882,63.294118,45,0.569708,73.827206,1.845680,0.265131,0.176279,0.729807,1.0,2.0,-1.0,1.313516,1.376789,4,2023_1104_1112,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,88.98,1.000000,0.941026,0.676923,0.351350,0.149377,0.085218,0.036114
1,0.8,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.52876,75.384559,1.884614,0.275133,0.17941,0.740689,0.400000,70.323529,68.5,2.617647,60.470588,62.0,47,75,14.235294,4.676471,58.294118,49,0.481441,71.490441,1.787261,0.220176,0.160698,0.680526,1.0,11.0,-10.0,1.313516,0.950975,12,2023_1104_1113,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,80.36,0.544272,0.183005,0.050300,0.019230,0.007441,0.001812,0.000619
2,0.8,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.52876,75.384559,1.884614,0.275133,0.17941,0.740689,0.250000,74.424242,76.0,7.030303,56.909091,56.0,48,70,13.424242,5.242424,55.757576,44,0.522086,71.236364,1.780909,0.270631,0.176877,0.736434,1.0,8.0,-7.0,1.313516,0.834293,10,2023_1104_1116,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,85.12,1.000000,0.516337,0.179549,0.086244,0.035891,0.013111,0.006306
3,0.8,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.52876,75.384559,1.884614,0.275133,0.17941,0.740689,0.333333,72.718750,72.0,5.656250,58.593750,59.0,47,73,14.187500,5.093750,56.437500,47,0.495432,70.496094,1.762402,0.255005,0.167288,0.687117,1.0,9.0,-8.0,1.313516,0.937026,11,2023_1104_1120,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,84.86,1.000000,0.579162,0.146244,0.082779,0.037729,0.017131,0.007497
4,0.8,82.176471,80.0,13.676471,62.205882,63.0,48,90,15.205882,5.058824,64.382353,53,0.52876,75.384559,1.884614,0.275133,0.17941,0.740689,0.333333,77.156250,77.0,6.906250,57.593750,56.5,50,72,14.500000,2.375000,56.187500,45,0.530645,69.980469,1.749512,0.279926,0.170526,0.667339,1.0,3.0,-2.0,1.313516,0.979970,5,2023_1104_1124,92.24,1.0,0.985207,0.811241,0.640182,0.436759,0.289307,0.146452,87.12,1.000000,0.889041,0.451510,0.241393,0.097918,0.057396,0.025092


In [105]:
man_test_tour.to_csv('man_test_tour_phase_1.csv', index = False)

# Woman

In [113]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'Seed1',
             'Seed2',
             'SeedDiff',
             'quality_march_T1',
             'quality_march_T2']

X_train = woman_train[to_select]
Y_train = woman_train['ResultDiff']

woman_test_tour =  woman_test[~(woman_test['Seed1'].isnull() | woman_test['Seed2'].isnull())].reset_index(drop = True)
woman_test_IDS = woman_test_tour['ID'] 
woman_test_tour = woman_test_tour[to_select]

In [114]:
#############
## XGBoost ##
#############

xgb_params = pd.read_csv('woman_XGB_Phase_1_42_Optuna_Hyperparameters.csv')

xgb_md = XGBRegressor(tree_method = 'hist', 
                      max_depth = xgb_params['max_depth'][0],
                      learning_rate = xgb_params['learning_rate'][0],
                      n_estimators = xgb_params['n_estimators'][0],
                      gamma = xgb_params['gamma'][0],
                      min_child_weight = xgb_params['min_child_weight'][0],
                      colsample_bytree = xgb_params['colsample_bytree'][0],
                      subsample = xgb_params['subsample'][0]).fit(X_train, Y_train)

xgb_pred = xgb_md.predict(woman_test_tour)

##############
## LightGBM ##
##############

lgb_params = pd.read_csv('woman_LightGBM_Phase_1_42_Optuna_Hyperparameters.csv')

lgb_md = LGBMRegressor(boosting_type = 'gbdt', 
                       n_estimators = lgb_params['n_estimators'][0],
                       learning_rate = lgb_params['learning_rate'][0],
                       max_depth = lgb_params['max_depth'][0],
                       lambda_l1 = lgb_params['lambda_l1'][0],
                       lambda_l2 = lgb_params['lambda_l2'][0],
                       num_leaves = lgb_params['num_leaves'][0],
                       bagging_fraction = lgb_params['bagging_fraction'][0],
                       feature_fraction = lgb_params['feature_fraction'][0]).fit(X_train, Y_train)

lgb_pred = lgb_md.predict(woman_test_tour)

##################
## HistGradient ##
##################

hist_params = pd.read_csv('woman_Hist_Phase_1_42_Optuna_Hyperparameters.csv')

hist_md = HistGradientBoostingRegressor(l2_regularization = hist_params['l2_regularization'][0],
                                        early_stopping = False,
                                        learning_rate = hist_params['learning_rate'][0],
                                        max_iter = hist_params['max_iter'][0],
                                        max_depth = hist_params['max_depth'][0],
                                        max_bins = hist_params['max_bins'][0],
                                        min_samples_leaf = hist_params['min_samples_leaf'][0],
                                        max_leaf_nodes = hist_params['max_leaf_nodes'][0]).fit(X_train, Y_train)

hist_pred = hist_md.predict(woman_test_tour)

##############
## Ensemble ##
##############

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
woman_test_tour['ResultDiff'] = np.round_(ens_pred, decimals = 0).astype(int)
woman_test_tour['ID'] = woman_test_IDS

[LightGBM] [Warning] lambda_l1 is set=9.27034195631365, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.27034195631365
[LightGBM] [Warning] feature_fraction is set=0.3451001288372496, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3451001288372496
[LightGBM] [Warning] lambda_l2 is set=1.7560660843788178, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7560660843788178
[LightGBM] [Warning] bagging_fraction is set=0.2077432535958427, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2077432535958427


In [115]:
data_538 = ['ID',
            'X1_team_rating',
            'X1_rd1_win',
            'X1_rd2_win',
            'X1_rd3_win',
            'X1_rd4_win',
            'X1_rd5_win',
            'X1_rd6_win',
            'X1_rd7_win',
            'X2_team_rating',
            'X2_rd1_win',
            'X2_rd2_win',
            'X2_rd3_win',
            'X2_rd4_win',
            'X2_rd5_win',
            'X2_rd6_win',
            'X2_rd7_win']

woman_test_538 = woman_test[data_538]

woman_test_tour = pd.merge(woman_test_tour, woman_test_538, on = 'ID', how = 'left')
woman_test_tour.head()

,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2,ResultDiff,ID,X1_team_rating,X1_rd1_win,X1_rd2_win,X1_rd3_win,X1_rd4_win,X1_rd5_win,X1_rd6_win,X1_rd7_win,X2_team_rating,X2_rd1_win,X2_rd2_win,X2_rd3_win,X2_rd4_win,X2_rd5_win,X2_rd6_win,X2_rd7_win
0,0.0,69.866667,69.0,10.1,57.066667,57.0,44,72,13.2,4.1,55.833333,44,0.517325,70.556667,1.763917,0.194598,0.211104,0.673198,0.000000,74.366667,77.5,9.566667,63.866667,64.5,54,78,14.633333,3.000000,56.866667,44,0.481218,74.895000,1.872375,0.205476,0.160871,0.673503,10.0,7.0,3.0,0.877673,0.863354,-3,2023_3104_3112,82.91,1.0,0.390015,0.019864,0.006137,0.001336,0.000273,0.000021,87.04,1.0,0.572748,0.087358,0.025801,9.917739e-04,2.489902e-04,7.604600e-05
1,0.0,69.866667,69.0,10.1,57.066667,57.0,44,72,13.2,4.1,55.833333,44,0.517325,70.556667,1.763917,0.194598,0.211104,0.673198,0.333333,72.000000,71.0,9.258065,63.064516,62.0,50,83,17.290323,2.967742,59.387097,43,0.483788,75.872581,1.896815,0.182095,0.199052,0.708221,10.0,7.0,3.0,0.877673,0.736285,-3,2023_3104_3124,82.91,1.0,0.390015,0.019864,0.006137,0.001336,0.000273,0.000021,86.60,1.0,0.609985,0.045417,0.019620,6.171927e-03,2.045400e-03,2.695491e-04
2,0.0,69.866667,69.0,10.1,57.066667,57.0,44,72,13.2,4.1,55.833333,44,0.517325,70.556667,1.763917,0.194598,0.211104,0.673198,1.000000,59.366667,58.0,3.866667,50.500000,50.0,43,61,10.866667,2.466667,52.533333,37,0.473866,65.320000,1.633000,0.234849,0.201812,0.734937,10.0,16.0,-6.0,0.877673,0.688312,17,2023_3104_3151,82.91,1.0,0.390015,0.019864,0.006137,0.001336,0.000273,0.000021,64.88,1.0,0.003967,0.000315,0.000008,1.712210e-07,4.785000e-09,4.500000e-11
3,0.0,69.866667,69.0,10.1,57.066667,57.0,44,72,13.2,4.1,55.833333,44,0.517325,70.556667,1.763917,0.194598,0.211104,0.673198,1.000000,74.500000,73.5,17.058824,59.705882,61.5,40,70,13.058824,4.000000,60.382353,42,0.481470,74.169118,1.854228,0.294608,0.194995,0.682838,10.0,13.0,-3.0,0.877673,0.840404,5,2023_3104_3156,82.91,1.0,0.390015,0.019864,0.006137,0.001336,0.000273,0.000021,77.35,1.0,0.101450,0.033836,0.002216,3.158956e-04,3.518961e-05,1.226490e-06
4,0.0,69.866667,69.0,10.1,57.066667,57.0,44,72,13.2,4.1,55.833333,44,0.517325,70.556667,1.763917,0.194598,0.211104,0.673198,0.500000,68.806452,71.0,10.193548,58.612903,57.0,51,68,14.838710,2.483871,55.838710,44,0.486013,71.066935,1.776673,0.207898,0.197780,0.676015,10.0,6.0,4.0,0.877673,1.094400,-6,2023_3104_3160,82.91,1.0,0.390015,0.019864,0.006137,0.001336,0.000273,0.000021,87.24,1.0,0.593834,0.147708,0.040496,8.219832e-03,6.977801e-04,2.162405e-04


In [116]:
man_test_tour.to_csv('woman_test_tour_phase_1.csv', index = False)